In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Descargar recursos necesarios
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/gugui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gugui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
#Función de preprocesamiento de texto

def preprocesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar las stopwords
    stop_words = set(stopwords.words('spanish')) 
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words]

    # Lematización
    lemmatizer = WordNetLemmatizer()
    palabras_lematizadas = [lemmatizer.lemmatize(palabra) for palabra in palabras]

    # Volver a unir las palabras lematizadas en un solo texto
    return ' '.join(palabras_lematizadas)


In [28]:
#importa JSON
import json

# Cargar el archivo JSON
with open("grafo_hierarquia.json", "r", encoding="utf-8") as file:
    grafo_json = json.load(file)

# Crear una lista de nodos con categorías, subcategorías y productos
nodos = []
for cultivo, data in grafo_json.items():
    nodos.append(cultivo)  # Agregar el nombre del cultivo
    if 'Productos' in data:
        for producto in data['Productos']:
            nodos.append(producto)  # Agregar cada producto


In [29]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo de embeddings de frases
model = SentenceTransformer('all-MiniLM-L6-v2')


In [30]:
import numpy as np

In [31]:
# Aplicar el preprocesamiento y convertir los nodos procesados a embeddings
nodos_procesados = [preprocesar_texto(nodo) for nodo in nodos]
nodos_embeddings = model.encode(nodos_procesados)
nodos_embeddings = np.array(nodos_embeddings)


In [32]:
import faiss
index = faiss.IndexFlatL2(nodos_embeddings.shape[1])
index.add(nodos_embeddings)


In [53]:
def consulta_rag(consulta, index, nodos, model, top_k=5):
    # Generar el embedding de la consulta
    consulta_embedding = model.encode([preprocesar_texto(consulta)])
    
    # Buscar en el índice los nodos más cercanos
    D, I = index.search(np.array(consulta_embedding), k=top_k)  
    
    # Mostrar los resultados
    resultados = []
    for i in range(top_k):
        nodo = nodos[I[0][i]]
        distancia = D[0][i]
        resultados.append((nodo, distancia))
        print(f"Resultado {i + 1}: {nodo} (Distancia: {distancia:.4f})")
    
    return resultados


In [45]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar el modelo y el tokenizador de GPT-2
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Ejemplo de uso
input_text = "Hola, ¿cómo estás?"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generar una predicción
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decodificar y mostrar el resultado
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Hola, ¿cómo estás?

I'm sorry, I'm sorry.

I'm sorry, I'm sorry.

I'm sorry, I'm sorry.

I'm sorry, I'm sorry


In [35]:
import torch

# Supongamos que consulta_embedding tiene una dimensión de 768
consulta_embedding = torch.tensor(consulta_embedding)

# Reducir la dimensión a 384
consulta_embedding_384 = consulta_embedding[:384].numpy()  # Tomar solo los primeros 384 elementos

print(f"Dimensión después de reducción manual: {consulta_embedding_384.shape}")


Dimensión después de reducción manual: (384,)


/tmp/ipykernel_49944/3978295796.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  consulta_embedding = torch.tensor(consulta_embedding)


In [47]:
# Definir el número de nodos más cercanos que deseas obtener
top_k = 5

# Buscar en el índice FAISS los 5 nodos más cercanos
D, I = index.search(np.array([consulta_embedding_384]), k=top_k)  # Buscar los 5 nodos más cercanos

# Mostrar los resultados
print(f"Consulta: {consulta}")
for i in range(top_k):
    print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")

# Concatenar los nodos más cercanos para formar un contexto
contexto = " ".join([nodos[I[0][i]] for i in range(top_k)])

# Tokenizar el contexto
input_ids = tokenizer.encode(contexto, return_tensors="pt")

# Generar una respuesta
outputs = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2)

# Decodificar y mostrar la respuesta generada
respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Respuesta generada: {respuesta}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Consulta: Spodoptera
Resultado 1: Imunit® (Distancia: 2517.7275)
Resultado 2: Vid (Distancia: 2521.0815)
Resultado 3: Kit Sistiva®  (Distancia: 2521.4136)
Resultado 4: Kit Sistiva®  (Distancia: 2521.4136)
Resultado 5: Bellis® (Distancia: 2521.8257)
Respuesta generada: Imunit® Vid Kit Sistiva®  Kit Sistiva®  Bellis® Viva™  Vivi® Sostiva  Sotiva ® Vivo®

Vivis™ VIVi™ Sustiva Vividiva
.
,
:
-
The Vixen® is a brand new product that is designed to be used with the VIXEN®. The VXVIVI® has been designed for use with VVIXE® and VXXV® products. VxVixE is an innovative and innovative product designed specifically for the use of VxxVX®, VxxxV, and XxxX. It is intended to provide a high quality, high performance, low cost, non-toxic, environmentally friendly, safe, portable, easy to use, affordable, convenient, reliable, durable, eco-friendly, ergonomic, light weight, lightweight, compact, versatile, efficient,


### Desde aca para adelante revisar!!!!

In [50]:
from transformers import BertModel, BertTokenizer
import torch

# Cargar el modelo y el tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Definir la consulta
consulta = "Rachiplusia Nu"  # O cualquier otro término que desees

# Tokenizar la consulta
inputs = tokenizer(consulta, return_tensors="pt")

# Obtener el embedding (la salida del modelo)
with torch.no_grad():
    outputs = model(**inputs)


In [51]:
# Tomar la representación del [CLS] token (generalmente el primero)
consulta_embedding_384 = outputs.last_hidden_state[:, 0, :].numpy()

In [52]:
# Asegurarte de que consulta_embedding_384 tenga la forma correcta (1, 384)
consulta_embedding_384 = np.expand_dims(consulta_embedding_384, axis=0)

# Ahora puedes continuar con la búsqueda en FAISS
top_k = 5
D, I = index.search(consulta_embedding_384, k=top_k)  # Buscar los 5 nodos más cercanos

# Mostrar los resultados
print(f"Consulta: {consulta}")
for i in range(top_k):
    print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")



ValueError: too many values to unpack (expected 2)

In [ ]:
# Concatenar los nodos más cercanos para formar un contexto
contexto = " ".join([nodos[I[0][i]] for i in range(top_k)])

# Tokenizar el contexto
input_ids = tokenizer.encode(contexto, return_tensors="pt")

# Generar una respuesta
outputs = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2)

# Decodificar y mostrar la respuesta generada
respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Respuesta generada: {respuesta}")
